In [1]:
import os
from dotenv import load_dotenv
from notion_client import Client
import json
import pandas as pd

In [2]:
load_dotenv()

True

In [3]:
notion_token = os.getenv("NOTION_TOKEN")
datasource_id = os.getenv("DATA_SOURCE_ID")
notion = Client(auth=notion_token)

In [4]:
response = notion.data_sources.query(data_source_id = datasource_id)

In [5]:
print(json.dumps(response.get('results')[0].get('properties'), indent = 2))
properties = response.get('results')[0].get('properties')

{
  "Date Started": {
    "id": "OQHx",
    "type": "date",
    "date": {
      "start": "2025-10-01",
      "end": null,
      "time_zone": null
    }
  },
  "Subject": {
    "id": "VhNW",
    "type": "select",
    "select": {
      "id": "134ec5c6-92b1-4d25-8099-67afa458ebaa",
      "name": "Field Recording",
      "color": "yellow"
    }
  },
  "Priority": {
    "id": "Xz%5De",
    "type": "select",
    "select": {
      "id": "<s~i",
      "name": "low",
      "color": "red"
    }
  },
  "Source": {
    "id": "%5D%3D%3CR",
    "type": "select",
    "select": {
      "id": "R^:J",
      "name": "Youtube",
      "color": "red"
    }
  },
  "Scope": {
    "id": "%5EKZ%3A",
    "type": "select",
    "select": {
      "id": "D>;f",
      "name": "Quick Win",
      "color": "green"
    }
  },
  "Status": {
    "id": "%5Ex%40G",
    "type": "status",
    "status": {
      "id": "831192fd-f990-4196-93ee-04400ddd5d40",
      "name": "Completed",
      "color": "green"
    }
  },
  "URL": {


In [6]:
print(properties.get('Title').get('title')[0].get('plain_text'))

KMRU Field Recording


In [7]:
def extract_value(prop):
    """Extrait la valeur brute selon le type de propriété Notion."""
    prop_type = prop.get('type')
    
    if prop_type == 'title':
        titles = prop.get('title', [])
        return titles[0].get('plain_text') if titles else None
    elif prop_type == 'date':
        date_data = prop.get('date')
        return date_data.get('start') if date_data else None
    elif prop_type == 'select':
        select_data = prop.get('select')
        return select_data.get('name') if select_data else None
    elif prop_type == 'status':
        status_data = prop.get('status')
        return status_data.get('name') if status_data else None
    elif prop_type == 'url':
        return prop.get('url')
    return None

# 1. Extraction des résultats
learnings = response.get('results', [])

# 2. Transformation en liste de dictionnaires (Aplatissage)
entries = []
for lesson in learnings:
    properties = lesson.get('properties', {})
    # Cette ligne crée un dictionnaire propre pour chaque ligne Notion
    row = {name: extract_value(obj) for name, obj in properties.items()}
    entries.append(row)

# 3. Création du DataFrame
df = pd.DataFrame(entries)

# Affichage du résultat
df.head()

,Date Started,Subject,Priority,Source,Scope,Status,URL,Topic,Title
0,2025-10-01,Field Recording,low,Youtube,Quick Win,Completed,https://www.youtube.com/watch?v=B4F5PTbki_A&li...,Music,KMRU Field Recording
1,None,Business Analytics,High,Book,Long,Not started,None,Data,The Saas PlayBook
2,None,Web Development,low,Youtube,Long,Not started,https://www.youtube.com/watch?v=c-hKSbzooAg&li...,Software Engineering,Google Drive Replica
3,None,AI,High,Youtube,Medium,Not started,https://www.youtube.com/watch?v=vzJOAnwIokM&li...,Data,LangChain Update
4,2025-12-10,Synthesis,Medium,Youtube,Medium,Completed,https://www.youtube.com/watch?v=QxyKeCl1PQk&t=...,Music,Moog Matriarch - Mega Tutorial
